# Predict Segmentation 

In [ ]:
import numpy as np
import random
import os
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import tensorflow as tf
import keras.backend as K
from keras.models import Model

from model_fcn import resnet50_fcn, resnet50_16s_fcn, resnet50_8s_fcn
from data_generator import seg_data_generator, pad_image
import utils

model_input_dir = '/work/fcn_models/segmentation_32s'
gpu_id = 0

img_dir = '/data/COCO/Segmentation/'
utils.config_tf()

df = pd.read_csv(img_dir + 'labels.txt',header=None)
n_classes = len(df.index)

net = "resnet50"

#from IPython.core.debugger import Tracer
#Tracer()()
# create model
#gpu = '/gpu:' + str(gpu_id)
#with tf.device(gpu):
if net == 'resnet50' : 
    model, stride = resnet50_fcn(n_classes)

if net == 'resnet50_16s' :
    model, stride = resnet50_16s_fcn(n_classes)

if net == 'resnet50_8s' :
    model, stride = resnet50_8s_fcn(n_classes) 

if model_input_dir != '':
    model.load_weights(model_input_dir + '/best_weights.hdf5')

In [ ]:
img_id = 0
n_rows = 5


# load image and predict
val_img_dir = img_dir + 'val_images/'
val_label_dir = img_dir + 'val_labels/'

img_list_val = os.listdir(val_img_dir)
img_list_val = img_list_val[img_id:]
pred_gen = seg_data_generator(stride,n_classes,val_img_dir,val_label_dir,img_list_val)
val_gen = seg_data_generator(stride,n_classes,val_img_dir,val_label_dir,img_list_val,preprocess = False)

# Visualise in tiled plot
fig = plt.figure(figsize=(15,15))

for idx in range(n_rows) :
	for row_img in range(2) :
		x_img, _ = val_gen.next()
		x_img = np.array(x_img,dtype='uint8')

		x, y = pred_gen.next()
		y = np.argmax(y,axis =-1)[0]

		pred = model.predict(x,batch_size=1)[0]
		pred = np.argmax(pred,axis =-1)

		# Visualise result
		fig.add_subplot(n_rows,6,idx*6 + row_img*3 +1)
		plt.imshow(x_img[0])
		
		fig.add_subplot(n_rows,6,idx*6 + row_img*3 + 2)
		plt.imshow(y, vmin=0,vmax=n_classes-1)
		
		fig.add_subplot(n_rows,6,idx*6 + row_img*3 + 3)
		plt.imshow(pred, vmin=0,vmax=n_classes-1)

plt.show()

# Predict Pose Estimation 

In [ ]:
import numpy as np
import random
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import keras.backend as K
from keras.models import Model

from model_fcn import resnet50_fcn, resnet50_16s_fcn, resnet50_8s_fcn
from data_generator import pose_data_generator, pad_image
import utils

model_input_dir = '/work/fcn_models/pose_estimation_32s'
gpu_id = 0

n_joints = 14

net = "resnet50"
root_dir = '/home/tdnguyen'
root_dir = ''
img_dir = root_dir + '/data/lsp_dataset/images/'

utils.config_tf()

# create model
gpu = '/gpu:' + str(gpu_id)
#with tf.device(gpu):
if net == 'resnet50' : 
    model, stride = resnet50_fcn(n_joints)

if net == 'resnet50_16s' :
    model, stride = resnet50_16s_fcn(n_joints)
    
if net == 'resnet50_8s' :
    model, stride = resnet50_8s_fcn(n_joints) 

if model_input_dir != '':
    #debugger.set_trace()
    model.load_weights(model_input_dir + '/best_weights.hdf5')

In [ ]:
img_id = 15
n_rows = 5
n_cols = 4

# load image and predict
N_train_img = 1000
img_dir = root_dir + '/data/lsp_dataset/images/'
label_file = root_dir + '/data/lsp_dataset/joints.mat'
img_list = range(1,2000)

pred_gen = pose_data_generator(stride,n_joints,img_dir,label_file,
							  img_list[N_train_img+img_id:])
val_gen = pose_data_generator(stride,n_joints,img_dir,label_file,
							  img_list[N_train_img+img_id:],
							  preprocess=False)

# Visualise in tiled plot
fig = plt.figure(figsize=(15,15))

for idx in range(n_rows) :
	for row_img in range(2) :
		x_img, _ = val_gen.next()
		
		x, y = pred_gen.next()
		y = np.argmax(y,axis =-1)[0]

		pred = model.predict(x,batch_size=1)[0]
		
		# Visualise result
		fig.add_subplot(n_rows,6,idx*6 + row_img*3 +1)
		plt.imshow(x_img[0])
		
		fig.add_subplot(n_rows,6,idx*6 + row_img*3 + 2)
		plt.imshow(y, vmin=0,vmax=n_joints-1)
			
		fig.add_subplot(n_rows,6,idx*6 + row_img*3 + 3)
		plt.imshow(pred[:,:,0], vmin=0,vmax=1)

plt.show()